# Welcome to the MScResearch Notebook (Prototype)

## Table of content:
0. [Getting Started](#1.-Getting-Started)

## 0 Getting started

In this notebook, we are going to be looking at applying the extractive and abstractive summarization models to existing datasets. Our first dataset if the CNN/DailyMail. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd 'drive/MyDrive/MScProject'

/content/drive/MyDrive/MScProject


In [ ]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,random,math
import pickle
import operator
import re
import torch
import more_itertools
from itertools import groupby
from operator import itemgetter
from pprint import pprint
import pickle

!pip install transformers
!!pip install datasets
!pip install sentencepiece
!pip install docx2txt
!pip install python-docx
!pip install rouge_score
!pip install evaluation
!pip install bert-extractive-summarizer
!pip install pytorch-transformers
!pip install evaluate
!pip install bert_score

KeyboardInterrupt: ignored

In [ ]:
from datasets import load_dataset
import transformers

from transformers import BertTokenizerFast

#spencer/dialogsum_reformat another dataset

dialogsum_data = load_dataset("knkarthick/dialogsum")

Using custom data configuration knkarthick--dialogsum-8300315d3067c408


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-8300315d3067c408/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dialogsum_data

In [ ]:
def show_samples(dataset, row_name, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> id: {example[row_name[0]]}'")
        print(f"'>> dialogue: {example[row_name[1]]}'")
        print(f"'>> summary: {example[row_name[2]]}'")
        print(f"'>> topic: {example[row_name[3]]}'")

show_samples(dialogsum_data,row_name=['id','dialogue', 'summary','topic'])

In [ ]:
dialogsum_data.set_format("pandas")
dialogsum_df = dialogsum_data["train"][:]

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data = dialogsum_df["dialogue"][0]
num_tokens = list(data.split(" "))#use this is you want to analyse data
number_of_sentences = sent_tokenize(data)

print('The number of tokens is',len(num_tokens))
print('The number of sentences is',len(number_of_sentences))

The number of tokens is 157
The number of sentences is 21


In [ ]:
data = dialogsum_df["summary"][10]
num_tokens = list(data.split(" "))#use this is you want to analyse data
number_of_sentences = sent_tokenize(data)

print('The number of tokens is',len(num_tokens))
print('The number of sentences is',len(number_of_sentences))

The number of tokens is 24
The number of sentences is 2


In [ ]:
dialogsum_df

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance
...,...,...,...,...
12455,train_12455,#Person1#: Excuse me. You are Mr. Green from M...,Tan Ling picks Mr. Green up who is easily reco...,pick up someone
12456,train_12456,#Person1#: Mister Ewing said we should show up...,#Person1# and #Person2# plan to take the under...,conference center
12457,train_12457,#Person1#: How can I help you today?\n#Person2...,#Person2# rents a small car for 5 days with th...,rent a car
12458,train_12458,#Person1#: You look a bit unhappy today. What'...,#Person2#'s mom lost her job. #Person2# hopes ...,job losing


In [ ]:
#random sample 200 articles
a=random.sample(range(0,12460), 200)
data_reference_article = dialogsum_data['train'][a]['dialogue']
data_reference_groudtruth = dialogsum_data['train'][a]["summary"]

# Create the pandas DataFrame with column name
df = pd.DataFrame(
    {'Dialogue': data_reference_article,
     'Groundtruth': data_reference_groudtruth
    })

In [ ]:
df

,Dialogue,Groundtruth
0,#Person1#: I'm glad we took the train. I don't...,#Person1# and #Person2# agree that it is bette...
1,#Person1#: Hello?\n#Person2#: Hi Steve. This i...,Mike phones Steve and they both feel bored. Th...
2,#Person1#: Good afternoon. madam. How can I he...,#Person2#'s cell phone was stolen and #Person2...
3,#Person1#: Thank your for calling World Airlin...,#Person2# calls World Airline to book a plane ...
4,#Person1#: Look at this newspaper article abou...,#Person1# and #Person2# are talking about psyc...
...,...,...
195,#Person1#: Excuse me. I'm looking for Blueming...,"#Person1#'s looking for Bluemingdails, and #Pe..."
196,#Person1#: Where do you see yourself three yea...,Miss Zhang tells #Person1# about her working p...
197,#Person1#: Why have you decided to change jobs...,#Person2# tells #Person1# #Person2# hopes to w...
198,#Person1#: Hello. Manchester 6739792.\n#Person...,#Person2# calls to find Nicolas but #Person1# ...


## 1 Downloading standard dataset 1 (CNN/DailyMail)

In [ ]:
from datasets import load_dataset
import transformers

from transformers import BertTokenizerFast

cnn_dailymail_data = load_dataset("cnn_dailymail", "3.0.0")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
cnn_dailymail_data

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
def show_samples(dataset, row_name, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Article: {example[row_name[0]]}'")
        print(f"'>> highlights: {example[row_name[1]]}'")
        print(f"'>> id: {example[row_name[2]]}'")

show_samples(cnn_dailymail_data,row_name=['article','highlights', 'id'])


'>> Article: By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall. The inquests have now opened into the deaths last Saturday, with investigators saying the three died along with the family's pet dog, of carbon monoxide poisoning from a cooker. Tragic: The inquests have opened into the deaths of three members of the same family who were found in their static caravan last weekend. John and Audrey Cook are pictured . Awful: The family died following carbon monoxide poisoning at this caravan at the Tremarle Home Park in Camborne, Cornwall . It is also believed there was no working carbon monoxide detec

In [ ]:
cnn_dailymail_data.set_format("pandas")
cnn_dailymail_df = cnn_dailymail_data["train"][:]

In [ ]:
cnn_dailymail_df["article"].value_counts()[:20]

SHANGHAI, China (CNN)  -- If it weren't for the Internet, Murong Xuecun might still be working as a sales manager at a car company in the southern Chinese city of Chengdu. That is what he was doing when he started writing his first novel on his office's online bulletin board system back in 2001. The Internet is increasingly being seen in China as a tool for literary empowerment, analysts say. Week by week when he got home from work, Murong would post new pieces to a story that painted a bleak yet honest picture of modern urban life in the city where he lived. It contained tales about sex, love, gambling and drugs and became so popular that it soon appeared on numerous other online forums. Today the 35-year-old is considered one of the most famous authors to have emerged in contemporary China. His debut work, "Leave Me Alone: A novel of Chengdu," has been read by millions of Chinese "netizens" -- steady Internet users -- and adapted for film and television and translated into German, Fr

In [ ]:
cnn_dailymail_df["article"][0]

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data = cnn_dailymail_df["article"][0]
num_tokens = list(data.split(" "))#use this is you want to analyse data
number_of_sentences = sent_tokenize(data)

print('The number of tokens is',len(num_tokens))
print('The number of sentences is',len(number_of_sentences))

The number of tokens is 456
The number of sentences is 24


In [ ]:
cnn_dailymail_df["highlights"][0]

"Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund ."

In [ ]:
data = cnn_dailymail_df["highlights"][0]
num_tokens = list(data.split(" "))#use this is you want to analyse data
number_of_sentences = sent_tokenize(data)

print('The number of tokens is',len(num_tokens))
print('The number of sentences is',len(number_of_sentences))

The number of tokens is 39
The number of sentences is 3


In [ ]:
def filter_articles(example):
  '''
  If a string is present in the dataset return the article
  '''
  if  '(Reuters)' in str(example['article']):
    return example['article']
  elif '(CNN)' in str(example['article']):
    return example['article']


In [ ]:
cnn_dailymail_data.reset_format()

In [ ]:
cnn_dailymail_filterdata = cnn_dailymail_data.filter(filter_articles)

Parameter 'function'=<function filter_articles at 0x7fb856594dd0> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/288 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
cnn_dailymail_filterdata

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 75617
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1189
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1067
    })
})

### 1.1 Abstractive summarization models

#### 1.1.1 Pegasus-Cnn_dailymail

In [ ]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer,AutoModelForSeq2SeqLM 
#  Load Model and Tokenize
model_name = 'google/pegasus-cnn_dailymail'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print('The max length of this model is',tokenizer.max_len_single_sentence)
data = cnn_dailymail_filterdata['train'][0]['article']

The max length of this model is 1023


In [ ]:
tokens_input = tokenizer.encode("summarize: " + data, return_tensors="pt", max_length=1024)
outputs = model.generate(tokens_input,max_length=1024, length_penalty=2.0, num_beams=4)
pprint(tokenizer.decode(outputs[0], skip_special_tokens=True))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('Harry Potter star Daniel Radcliffe gains access to a reported £20 million '
 'fortune.<n>The young actor says he has no plans to fritter his cash '
 "away.<n>Radcliffe's earnings from the first five Potter films have been held "
 'in a trust fund.<n>His latest outing as the boy wizard in "Harry Potter and '
 'the Order of the Phoenix" is breaking records.')


In [ ]:
from datasets import load_metric
from evaluate import load

rouge_score = load_metric("rouge")

generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
reference_summary = cnn_dailymail_df["highlights"][0]

scores = rouge_score.compute(predictions=[generated_summary], references=[reference_summary])

print('rouge1:',scores["rouge1"].mid,'\n'
      'rouge2:',scores["rouge2"].mid,'\n'
      'rougeL:',scores["rougeL"].mid,'\n'
      'rougeLsum:',scores["rougeLsum"].mid,'\n')

#https://pypi.org/project/bert-score/
bertscore = load("bertscore")
results = bertscore.compute(predictions=[generated_summary], references=[reference_summary], lang="en")
pprint(results)

rouge1: Score(precision=0.515625, recall=0.8461538461538461, fmeasure=0.6407766990291262) 
rouge2: Score(precision=0.4126984126984127, recall=0.6842105263157895, fmeasure=0.5148514851485149) 
rougeL: Score(precision=0.5, recall=0.8205128205128205, fmeasure=0.6213592233009708) 
rougeLsum: Score(precision=0.515625, recall=0.8461538461538461, fmeasure=0.6407766990291262) 



Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

{'f1': [0.9235020875930786],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.11(hug_trans=4.20.1)',
 'precision': [0.9040862321853638],
 'recall': [0.9437701106071472]}


#### 1.1.2 Facebook/bart-large-cnn


In [ ]:
#  Load Model and Tokenize
model_name = 'facebook/bart-large-cnn'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print('The max length of this model is',tokenizer.max_len_single_sentence)

The max length of this model is 1022


In [ ]:
tokens_input = tokenizer.encode("summarize: " + data, return_tensors="pt", max_length=1024)
outputs = model.generate(tokens_input,max_length=1024, length_penalty=2.0, num_beams=4)
pprint(tokenizer.decode(outputs[0], skip_special_tokens=True))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a '
 "reported £20 million ($41.1 million) fortune. Radcliffe's earnings from the "
 "first five Potter films have been held in a trust fund. Details of how he'll "
 'mark his landmark birthday are under wraps.')


In [ ]:
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
reference_summary = cnn_dailymail_df["highlights"][0]

scores = rouge_score.compute(predictions=[generated_summary], references=[reference_summary])

print('rouge1:',scores["rouge1"].mid,'\n'
      'rouge2:',scores["rouge2"].mid,'\n'
      'rougeL:',scores["rougeL"].mid,'\n'
      'rougeLsum:',scores["rougeLsum"].mid,'\n')

bertscore = load("bertscore")
results = bertscore.compute(predictions=[generated_summary], references=[reference_summary], model_type=model_name ,lang="en")
pprint(results)

rouge1: Score(precision=0.5510204081632653, recall=0.6923076923076923, fmeasure=0.6136363636363635) 
rouge2: Score(precision=0.3333333333333333, recall=0.42105263157894735, fmeasure=0.372093023255814) 
rougeL: Score(precision=0.4897959183673469, recall=0.6153846153846154, fmeasure=0.5454545454545454) 
rougeLsum: Score(precision=0.5102040816326531, recall=0.6410256410256411, fmeasure=0.5681818181818182) 

{'f1': [0.7241030931472778],
 'hashcode': 'facebook/bart-large-cnn_L10_no-idf_version=0.3.11(hug_trans=4.20.1)',
 'precision': [0.7433146238327026],
 'recall': [0.7058596611022949]}


#### 1.1.3 t5-large

In [ ]:
#  Load Model and Tokenize
model_name = 't5-large'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print('The max length of this model is',tokenizer.max_len_single_sentence)

The max length of this model is 511


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
tokens_input = tokenizer.encode("summarize: " + data, return_tensors="pt", max_length=1024)
outputs = model.generate(tokens_input,max_length=1024, length_penalty=2.0, num_beams=4)
pprint(tokenizer.decode(outputs[0], skip_special_tokens=True))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('Harry Potter star Daniel Radcliffe turns 18 on monday. the young actor says '
 'he has no plans to fritter his cash away. he will be able to gamble in a '
 "casino, buy a drink in a pub. details of how he'll mark his landmark "
 'birthday are under wraps.')


In [ ]:
from datasets import load_metric

rouge_score = load_metric("rouge")

generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
reference_summary = cnn_dailymail_df["highlights"][0]

scores = rouge_score.compute(predictions=[generated_summary], references=[reference_summary])

print('rouge1:',scores["rouge1"].mid,'\n'
      'rouge2:',scores["rouge2"].mid,'\n'
      'rougeL:',scores["rougeL"].mid,'\n'
      'rougeLsum:',scores["rougeLsum"].mid,'\n')

bertscore = load("bertscore")
#results = bertscore.compute(predictions=[generated_summary], references=[reference_summary], lang="en")
#pprint(results)

results = bertscore.compute(predictions=[generated_summary], references=[reference_summary], model_type=model_name ,lang="en")
pprint(results)

rouge1: Score(precision=0.4489795918367347, recall=0.5641025641025641, fmeasure=0.5) 
rouge2: Score(precision=0.3333333333333333, recall=0.42105263157894735, fmeasure=0.372093023255814) 
rougeL: Score(precision=0.42857142857142855, recall=0.5384615384615384, fmeasure=0.4772727272727273) 
rougeLsum: Score(precision=0.42857142857142855, recall=0.5384615384615384, fmeasure=0.4772727272727273) 



/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


## 2 Downloading standard dataset 2 (Mediasum)

In [ ]:
from datasets import load_dataset
import transformers

from transformers import BertTokenizerFast

#spencer/dialogsum_reformat another dataset

dialogsum_data = load_dataset("knkarthick/dialogsum")

Using custom data configuration knkarthick--dialogsum-259845a8a5a4b7bc
Reusing dataset csv (/root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-259845a8a5a4b7bc/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dialogsum_data

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [ ]:
def show_samples(dataset, row_name, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> id: {example[row_name[0]]}'")
        print(f"'>> dialogue: {example[row_name[1]]}'")
        print(f"'>> summary: {example[row_name[2]]}'")
        print(f"'>> topic: {example[row_name[3]]}'")

show_samples(dialogsum_data,row_name=['id','dialogue', 'summary','topic'])

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-259845a8a5a4b7bc/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-e3616d861397a96d.arrow



'>> id: train_6155'
'>> dialogue: #Person1#: Hello, Anna speaking!
#Person2#: Hey, Anna, this is Jason.
#Person1#: Jason, where have you been hiding lately? You know it's been a long time since your last call. Have you been good?
#Person2#: Yes. How are you, Anna?
#Person1#: I am fine. What have you been doing?
#Person2#: Working. I've been really busy these days. I got a promotion.
#Person1#: That's great, congratulations!
#Person2#: Thanks. I am feeling pretty good about myself too. You know, bigger office, a raise and even an assistant.
#Person1#: That's good. So I guess I'll have to make an appointment to see you.
#Person2#: You are kidding.
#Person1#: How long have you been working there?
#Person2#: A bit over two years. This is a fast-moving company, and seniority isn't the only factor in deciding promotions.
#Person1#: How do you like your new boss?
#Person2#: She is very nice and open-minded.
#Person1#: Much better than the last one, huh?
#Person2#: Yeah. He was a real slave d

In [ ]:
dialogsum_data.set_format("pandas")
dialogsum_df = dialogsum_data["train"][:]

In [ ]:
dialogsum_df =  dialogsum_data["train"][:][['dialogue','summary']]
dialogsum_df.head()

,dialogue,summary
0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw..."
1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...
2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...
3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...
4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...


In [ ]:
#  Load Model and Tokenize
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/bart-base-cnn")
model = BartForConditionalGeneration.from_pretrained("ainize/bart-base-cnn")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
inputs = tokenizer.encode("summarize: " + dialogsum_df['dialogue'][2], return_tensors="pt", max_length=512)
outputs_dialogue = model.generate(inputs, max_length=150, min_length=18, length_penalty=1.0, num_beams=4, early_stopping=True)
tokenizer.decode(outputs_dialogue[0], skip_special_tokens=True)

'"I didn\'t see them. What a shame!"\n"I\'d like to help you look for the missing keys"'

In [ ]:
dialogsum_df["summary"][2]

"#Person1#'s looking for a set of keys and asks for #Person2#'s help to find them."

In [ ]:
from datasets import load_metric

rouge_score = load_metric("rouge")

generated_summary = tokenizer.decode(outputs_dialogue[0], skip_special_tokens=True)
scores = rouge_score.compute(predictions=[generated_summary], references=[reference_summary])
scores

{'rouge1': AggregateScore(low=Score(precision=0.2631578947368421, recall=0.10869565217391304, fmeasure=0.15384615384615385), mid=Score(precision=0.2631578947368421, recall=0.10869565217391304, fmeasure=0.15384615384615385), high=Score(precision=0.2631578947368421, recall=0.10869565217391304, fmeasure=0.15384615384615385)),
 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
 'rougeL': AggregateScore(low=Score(precision=0.15789473684210525, recall=0.06521739130434782, fmeasure=0.0923076923076923), mid=Score(precision=0.15789473684210525, recall=0.06521739130434782, fmeasure=0.0923076923076923), high=Score(precision=0.15789473684210525, recall=0.06521739130434782, fmeasure=0.0923076923076923)),
 'rougeLsum': AggregateScore(low=Score(precision=0.21052631578947367, recall=0.08695652173913043, fmeasure=0.12307692307692308), mid=Score(precision=0.210526315789473

## 3 Example of a longformer 

In [ ]:
from transformers import AutoTokenizer, LongT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")
model = LongT5ForConditionalGeneration.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")
text = dialogsum_df['dialogue'][2]
input_ids = tokenizer("summarize: " + text, return_tensors="pt").input_ids  # Batch size 1
outputs = model.generate(input_ids,max_length=40,no_repeat_ngram_size=7,num_beams=2,early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:770: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


In [ ]:
pprint(summary)

('a pregnancy has been reported in the past few years, but it is still unclear '
 'whether or not she is pregnant or pregnant. in this article, we discuss the '
 'pregnancy rumors')


In [ ]:
from datasets import load_metric

rouge_score = load_metric("rouge")

generated_summary = tokenizer.decode(outputs_dialogue[0], skip_special_tokens=True)
scores = rouge_score.compute(predictions=[generated_summary], references=[reference_summary])
scores

In [ ]:
from transformers import LEDTokenizer, LEDForConditionalGeneration
# Load model and tokenizer
model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv")
tokenizer = LEDTokenizer.from_pretrained("allenai/led-large-16384-arxiv")

# Tokenize and summarize
inputs = tokenizer.encode(text, return_tensors="pt")
# Global attention on the first token (cf. Beltagy et al. 2020)
global_attention_mask = torch.zeros_like(inputs)
global_attention_mask[:, 0] = 1
# Generate Summary
summary_ids = model.generate(inputs, global_attention_mask=global_attention_mask,max_length=40, no_repeat_ngram_size=7,num_beams=2,early_stopping=True)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

In [ ]:
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
# Print summary
pprint(summary)

(' Beyonce has yet to confirm the rumors that she’s pregnant with her second '
 'child, but she could already be eating for two. after last night’s concert '
 'in Dublin, Ireland')


# References

https://huggingface.co/course/chapter7/5?fw=tf

https://huggingface.co/docs/transformers/tasks/summarization

https://huggingface.co/ainize/bart-base-cnn

https://huggingface.co/course/chapter7/5?fw=tf

https://colab.research.google.com/drive/1Ekd5pUeCX7VOrMx94_czTkwNtLN32Uyu?usp=sharing

https://rubikscode.net/2022/04/25/text-summarization-with-huggingface-transformers/

https://pypi.org/project/bert-extractive-summarizer/

https://huggingface.co

https://huggingface.co/datasets/spencer/dialogsum_reformat

https://github.com/cylnlp/dialogsum/blob/main/DialogSum_Data/dialogsum.test.jsonl